In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "수목관리업무매뉴얼(표지제외).pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

207


In [6]:
print(f"{docs[117].page_content[:200]}\n")
print(docs[0].metadata)

수목 관리 업무매뉴얼
112
나) 수목용 비료의 종류 
(1) 단일 성분 비료 
(가) 질소질 비료는 질소를 주요 성분으로 보증하는 비료로 조경수의 생육 초기
에 사용되며, 주로 사용하는 비료는 질소질이 45% 함유된 요소와 20% 함
유된 황산암모늄 등이 있다. 
(나) 인산질 비료는 인산을 주요 성분으로 보증하는 비료로 수목의 뿌리 발근을 
향상시키기 

{'producer': 'Acrobat Distiller 10.1.16 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2022-12-23T14:12:34+09:00', 'author': 'YoungJa70', 'moddate': '2022-12-26T18:09:18+09:00', 'title': '<283232313231352920BCF6B8F1B0FCB8AE20BEF7B9ABB8C5B4BABEF32E687770>', 'source': '수목관리업무매뉴얼(표지제외).pdf', 'total_pages': 207, 'page': 0, 'page_label': '1'}


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

303


In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="embeddinggemma:300m")

c:\Users\Bistelligence\AppData\Local\pypoetry\Cache\virtualenvs\ollama-app-PvdVXOUT-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[-0.19646348, 0.022563307, 0.04347242, 0.043465137, -0.07891526, 0.011886077, -0.048882663, 0.015704768, 0.037029132, -0.055120196]


In [10]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

ids = vector_store.add_documents(documents=all_splits)

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[117])

page_content='속   표   지' metadata={'producer': 'Acrobat Distiller 10.1.16 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2022-12-23T14:12:34+09:00', 'author': 'YoungJa70', 'moddate': '2022-12-26T18:09:18+09:00', 'title': '<283232313231352920BCF6B8F1B0FCB8AE20BEF7B9ABB8C5B4BABEF32E687770>', 'source': '수목관리업무매뉴얼(표지제외).pdf', 'total_pages': 207, 'page': 0, 'page_label': '1', 'start_index': 0}
